In [16]:
import os
import sys
import torch
import torch.nn.functional as F
import torchaudio
import speechbrain as sb
import speechbrain.nnet.schedulers as schedulers
import numpy as np
from tqdm import tqdm
import logging
from hyperpyyaml import load_hyperpyyaml
import csv
device = torch.device('cuda')
logger = logging.getLogger(__name__)

# from mir_eval.separation import bss_eval_sources
from speechbrain.dataio.dataio import read_audio
from fast_bss_eval import bss_eval_sources
import librosa

In [17]:
import time
def append_audio(upsampled, item_name, file_name, out_est, out_target,out_mix, audio):
   if upsampled:
      if "source" in file_name and "hat_up" not in file_name :
         out_target[item_name] =out_target[item_name]+[(audio)] if item_name in out_target else [audio]
         return
      elif "hat_up" in file_name:
         out_est[item_name] =out_est[item_name]+[(audio)] if item_name in out_est else [audio]
         return
      elif "mix" in file_name:
         out_mix.append(audio)
   if not upsampled:
      if "source" in file_name and "hat" not in file_name:
         out_target[item_name] =out_target[item_name]+[(audio)] if item_name in out_target else [audio]
         return
      elif "hat" in file_name and "hat_up" not in file_name:
         out_est[item_name] = out_est[item_name]+[(audio)] if item_name in out_est  else [audio]
         return
      elif "mix" in file_name:
         out_mix.append(audio)
         

def read_all_audio(path, upsampled=False):
   
   audio_ids = []
   out_target_orig = {}
   out_target_pred = {}
   out_est = {}
   out_upsampled = {}
   out_mix = []
   all_files = os.listdir(path)[:70]
   all_files = os.listdir(path)
   for file_name in tqdm(all_files, total=len(all_files)) :
      filepath = os.path.join(path, file_name)
      item_name = file_name.split('_')[0]
      if upsampled and "source" in file_name and "hat" not in file_name:
         continue
      audio, sr_ = librosa.load(filepath, sr=8000)
      audio = torch.from_numpy(audio).unsqueeze(0)
      append_audio(upsampled, item_name, file_name, out_est, out_target,out_mix, audio)
   audio_ids = list(out_target.keys()) 
   out_target = list(out_target.values())
   out_est = list(out_est.values())
   return audio_ids, out_target, out_est, out_mix

audio_ids,out_target, out_est, out_mix = read_all_audio('results/audio_results', True)

 82%|████████▏ | 17275/21000 [07:03<01:31, 40.84it/s]


KeyboardInterrupt: 

In [ ]:
# orig_audio, sr_orig = librosa.load('results/audio_results/item0_source1.wav',)
# hat_audio, sr_hat = librosa.load('results/audio_results/item0_source1hat.wav',8000)
# hat_up_audio, sr_hat_up = librosa.load('results/audio_results/item0_source1hat_up.wav',)
# orig_audio.shape, sr_orig, hat_audio.shape, sr_hat, hat_up_audio.shape, sr_hat_up

In [ ]:
len(audio_ids),len(out_target), len(out_est) , len(out_mix)

(3000, 3000, 3000, 3000)

In [ ]:
audio_ids[0], out_target[0], out_est[0], out_mix[0]

('item0',
 [tensor([[ 0.0006,  0.0009,  0.0007,  ...,  0.0009,  0.0001, -0.0002]]),
  tensor([[-0.0013, -0.0022, -0.0013,  ..., -0.0037, -0.0023, -0.0005]])],
 [tensor([[-0.0033, -0.0048,  0.0008,  ...,  0.0029, -0.0053,  0.0002]]),
  tensor([[-0.0056, -0.0062, -0.0009,  ..., -0.0041, -0.0057, -0.0012]])],
 tensor([[-0.0019, -0.0030, -0.0024,  ..., -0.0047, -0.0019,  0.0015]]))

In [ ]:
class Separation(sb.Brain):
   def compute_objectives(self, predictions, targets):
        """Computes the si-snr loss"""
        return self.hparams.loss(targets, predictions)
     
   def get_metrics(self,audio_ids, targets, preds, mixtures):

        # Create folders where to store audio
      save_file = os.path.join(self.hparams.output_folder, "test_results.csv")

        # Variable init

      all_sdrs = []
      all_sdrs_i = []
      all_sisnrs = []
      all_sisnrs_i = []
      csv_columns = ["snt_id", "sdr", "sdr_i", "si-snr", "si-snr_i"]

        
      
      with open(save_file, "w") as results_csv:
         writer = csv.DictWriter(results_csv, fieldnames=csv_columns)
         writer.writeheader()
            
         for audio_id, target, pred,mixture_ in tqdm(zip(audio_ids,targets, preds,mixtures), total=len(targets)):
            target = torch.cat(
            [target[i].unsqueeze(-1) for i in range(self.hparams.num_spks)],
            dim=-1,
        ).to(self.device)
            
            pred = torch.cat(
            [pred[i].unsqueeze(-1) for i in range(self.hparams.num_spks)],
            dim=-1,
        ).to(self.device)
  
            sisnr = self.compute_objectives(pred, target)
            # COmpute SI-SNR Improvement
            mixture_signal = torch.stack(
               [mixture_] * self.hparams.num_spks, dim=-1
            )
            mixture_signal = mixture_signal.to(target.device)
            sisnr_baseline = self.compute_objectives(
               mixture_signal, target
            )
            sisnr_i = sisnr.cpu().numpy() - sisnr_baseline.cpu().numpy()
  # Compute SDR
            sdr, _, _, _ = bss_eval_sources(
               target[0].t(),
               pred[0].t(),
            )

            sdr_baseline, _, _, _ = bss_eval_sources(
               target[0].t(),
               mixture_signal[0].t(),
            )
            
            sdr = sdr.cpu().numpy()
            sdr_baseline = sdr_baseline.cpu().numpy()
           
            sdr_i = sdr.mean() - sdr_baseline.mean()

            # Saving on a csv file
            row = {
               "snt_id": audio_id,
               "sdr": sdr.mean(),
               "sdr_i": sdr_i,
               "si-snr": -sisnr.item(),
               "si-snr_i": -sisnr_i.item(),
            }
            writer.writerow(row)

            # Metric Accumulation
            all_sdrs.append(sdr.mean())
            all_sdrs_i.append(sdr_i.mean())
            all_sisnrs.append(-sisnr.item())
            all_sisnrs_i.append(-sisnr_i.item())
      # logger.info("Mean SISNR is {}".format(np.array(all_sisnrs).mean()))
      # logger.info("Mean SISNRi is {}".format(np.array(all_sisnrs_i).mean()))
      # logger.info("Mean SDR is {}".format(np.array(all_sdrs).mean()))
      # logger.info("Mean SDRi is {}".format(np.array(all_sdrs_i).mean()))   


In [ ]:
out_target[0][0].shape, out_est[0][0].shape

(torch.Size([1, 35160]), torch.Size([1, 35160]))

In [ ]:
def run():
    # Load hyperparameters file with command-line overrides
   hparams_file, run_opts, overrides = sb.parse_arguments(["hyperparams.yaml"])
   hparams_file = 'hyperparams.yaml'
#    run_opts = {"device": "cuda:0"}
   with open(hparams_file) as fin:
      hparams = load_hyperpyyaml(fin, overrides)
        
      # Load pretrained model if pretrained_separator is present in the yaml
   if "pretrained_separator" in hparams:
       # run_on_main(hparams["pretrained_separator"].collect_files)
       hparams["pretrained_separator"].load_collected(
            device=run_opts["device"]
       )
   # Brain class initialization
   separator = Separation(
        modules=hparams["modules"],
        run_opts={"device": "cuda"},
        hparams=hparams,
    )
   separator.get_metrics(audio_ids, out_target, out_est, out_mix)
run()

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
100%|██████████| 3000/3000 [02:41<00:00, 18.63it/s]


### Retrieves the worst, median and best separation audio mixtures

In [2]:
import numpy as np
import pandas as pd

# Get worst
def get_worst(column,df):
   return df.loc[df[column].idxmin()]
# Get best 
def get_best(column,df):
   return df.loc[df[column].idxmax()]
# Get Median
def get_median(column,df):
   df.loc[df[column]== df[column].median()]
   df.sort_values(by=column, inplace=True)
   return df[df[column] < df[column].median()].iloc[-1]


def print_audio_info(stat, df):
   print(f"status: {stat} | sdr: {df['sdr']} | sdr_i: {df['sdr_i']} | si-snr: {df['si-snr']} | si-snr_i: {df['si-snr_i']}")
status = ['Worst', 'Median', 'Best']

# configs = ["no_noise", "no_noise_speedperturb","w_noise_speedperturb", "w_noise_wavedrop"]
configs = ["no_noise_speedperturb"]
for config in configs:
   df = pd.read_csv(f'results/upsampled_metrics/{config}.csv')
   df.replace([np.inf, -np.inf], np.nan, inplace=True)
# drop rows with NaNs
   df.dropna(inplace=True)
   # df.replace()
   print(f"------------CONFIG: {config}")
   print_audio_info(status[0], get_worst('si-snr',df))
   print_audio_info(status[1], get_median('si-snr',df))
   print_audio_info(status[2], get_best('si-snr',df))
   mean_values = df.loc[:, df.columns != 'snt_id'].mean()
   print(f'mean_values {mean_values}')

------------CONFIG: no_noise_speedperturb
status: Worst | sdr: 7.472182 | sdr_i: 0.14493132 | si-snr: 4.14491081237793 | si-snr_i: -2.8795289993286133
status: Median | sdr: 21.159967 | sdr_i: 20.366205 | si-snr: 14.19557285308838 | si-snr_i: 13.583800315856934
status: Best | sdr: 27.271698 | sdr_i: 25.772282 | si-snr: 20.748476028442383 | si-snr_i: 19.38570022583008
mean_values sdr         20.395723
sdr_i       18.038166
si-snr      13.974293
si-snr_i    11.997022
dtype: float64


In [ ]:
df = df.sort_values(by='snt_id')
df.head()

,snt_id,sdr,sdr_i,si-snr,si-snr_i
0,item0,27.312160,26.274744,16.086637,15.195269
1111,item1,20.704279,15.591076,11.947648,7.364430
111,item10,27.709038,25.243404,15.365202,13.305538
11,item100,20.437866,17.912374,14.745030,12.808769
1,item1000,22.681060,20.429329,15.974695,13.866444


In [ ]:
# import numpy as np
# import librosa

# # Load the original audio signal
# audio_file_orig = 'original_audio.wav'
# y_orig, sr_orig = librosa.load(audio_file_orig)

# # Load the upsampled audio signal
# audio_file_upsamp = 'upsampled_audio.wav'
# y_upsamp, sr_upsamp = librosa.load(audio_file_upsamp)

# # Compute the SNR of the upsampled audio signal compared to the original signal
# diff = np.sum((y_orig - y_upsamp)**2)
# snr = 10 * np.log10(np.sum(y_orig**2) / diff)

# print('SNR: {:.2f} dB'.format(snr))


SNR: 16.21 dB


In [ ]:
import librosa

# Load the original audio signal
audio_file_orig = 'original_audio.wav'
y_orig, sr_orig = librosa.load(audio_file_orig)

# Load the upsampled audio signal
audio_file_upsamp = 'upsampled_audio.wav'
y_upsamp, sr_upsamp = librosa.load(audio_file_upsamp)

# Compute the spectral centroid and rolloff of the original and upsampled audio signals
spec_centroid_orig = librosa.feature.spectral_centroid(y_orig, sr=sr_orig)[0]
spec_rolloff_orig = librosa.feature.spectral_rolloff(y_orig, sr=sr_orig)[0]

spec_centroid_upsamp = librosa.feature.spectral_centroid(y_upsamp, sr=sr_upsamp)[0]
spec_rolloff_upsamp = librosa.feature.spectral_rolloff(y_upsamp, sr=sr_upsamp)[0]

# Compute the mean and standard deviation of the spectral features
centroid_diff = spec_centroid_orig.mean() - spec_centroid_upsamp.mean()
rolloff_diff = spec_rolloff_orig.mean() - spec_rolloff_upsamp.mean()

centroid_std_diff = spec_centroid_orig.std() - spec_centroid_upsamp.std()
rolloff_std_diff = spec_rolloff_orig.std() - spec_rolloff_upsamp.std()

# Print the results
print('Spectral centroid difference: {:.2f}'.format(centroid_diff))
print('Spectral rolloff difference: {:.2f}'.format(rolloff_diff))

print('Spectral centroid std difference: {:.2f}'.format(centroid_std_diff))
print('Spectral rolloff std difference: {:.2f}'.format(rolloff_std_diff))


Spectral centroid difference: -215.47
Spectral rolloff difference: -267.58
Spectral centroid std difference: -97.98
Spectral rolloff std difference: -108.21


In [ ]:
sr_upsamp, sr_orig

(22050, 22050)